In [ ]:
def parse_sid(sid_text):
    pattern = re.compile(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)")
    sid = []
    for line in sid_text.splitlines():
        match = pattern.match(line.strip())
        if match:
            sender = match.group(1).strip()
            receiver = match.group(2).strip()
            message = match.group(3).strip()
            sid.append((f"{sender} -> {receiver}", message))
    return sid

In [ ]:
def extract_sbd_section(full_text):
    lines = full_text.splitlines()
    sbd_start = None
    for i, line in enumerate(lines):
        if "### Step 5" in line:
            sbd_start = i + 1
            break
    if sbd_start is None:
        print("No SBD section found!")
        return ""

    sbd_lines = []
    for line in lines[sbd_start:]:
        if line.strip().startswith("### Explanation"):
            break
        sbd_lines.append(line)

    return "\n".join(sbd_lines).strip()

def parse_sbd(sbd_text):
    
    lines = sbd_text.splitlines()

    sbd = {}
    current_actor = None
    states = {}
    current_state = None
    current_state_num = None

    actor_header_re = re.compile(r"^\s*####\s*(.+):$")
    state_header_re = re.compile(r"(\d+)\.\s+(\w+State):\s*(.*)")

    for idx, line in enumerate(lines):
        line = line.rstrip()
        if not line:
            continue

        # Detect actor header like: #### Customer:
        actor_match = actor_header_re.match(line)
        if actor_match:
            if current_actor:
                sbd[current_actor] = list(states.values())
            current_actor = actor_match.group(1).strip()
            states = {}
            current_state = None
            current_state_num = None
            continue

        if current_actor is None:
            continue  # Skip anything before the first actor header

        # Detect numbered state line like: 1. StartState:
        state_match = state_header_re.match(line)
        if state_match:
            current_state_num = int(state_match.group(1))
            state_type = state_match.group(2)
            description = state_match.group(3).strip()
            current_state = {
                "num": current_state_num,
                "type": state_type,
                "description": description
            }
            states[current_state_num] = current_state
            continue

        if current_state is None:
            continue

        line_stripped = line.strip()

        # Capture From/To/Msg in SendState, ReceiveState, and StartState
        if current_state["type"] in ["SendState", "StartState"]:
            if line_stripped.startswith("To:"):
                current_state["To"] = line.split(":", 1)[1].strip()
            elif line_stripped.startswith("Msg:"):
                current_state["Msg"] = line.split(":", 1)[1].strip()

        if current_state["type"] in ["ReceiveState", "StartState"]:
            if line_stripped.startswith("From:"):
                current_state["From"] = line.split(":", 1)[1].strip()
            elif line_stripped.startswith("Msg:"):
                current_state["Msg"] = line.split(":", 1)[1].strip()

        # Capture Description or Action for DoState and StartState
        if current_state["type"] in ["DoState", "StartState"]:
            if line_stripped.startswith("Description:"):
                current_state["Description"] = line.split(":", 1)[1].strip()

    if current_actor:
        sbd[current_actor] = list(states.values())

    return sbd

In [ ]:
##############SID Graph##########################
import os
os.environ["PATH"] += os.pathsep + r"C:\Users\qwerz\Documents\Privat\WWU\Wirtschaftsinformatik\Semester 7\Bachelorarbeit\PASS Diagrams\Diagram\llama"

from graphviz import Digraph
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")  # Make sure this is installed

# Define canonical entities
canonical_entities = [
    "Customer",
    "E-commerce Website"    
]

# Convert to spaCy Docs for similarity comparison
canonical_docs = {ent: nlp(ent) for ent in canonical_entities}

def normalize_entity(entity):
    entity_doc = nlp(entity)
    best_match = None
    best_score = 0.75  # Similarity threshold

    for canon_name, canon_doc in canonical_docs.items():
        score = entity_doc.similarity(canon_doc)
        if score > best_score:
            best_match = canon_name
            best_score = score

    return best_match if best_match else entity.strip()

def draw_sid_graph(sid_list, output_path="sid_graph"):
    dot = Digraph(comment="SID - Sequence Interaction Diagram")
    dot.attr(dpi='300')  # increase DPI for sharper text
    dot.attr(fontsize='14')  
    dot.attr(rankdir='LR',splines='polyline')  # Left to right direction
    
    # Deduplicate using a set
    normalized_edges = set()

    for interaction, action in sid_list:
        source, target = [normalize_entity(s.strip()) for s in interaction.split("->")]
        key = (source, target, action.strip())

        if key in normalized_edges:
            print(f"Skipping duplicate: {source} -> {target} [{action}]")
            continue

        normalized_edges.add(key)
        dot.node(source, source, shape="box", style="filled", fillcolor="lightblue")
        dot.node(target, target, shape="box", style="filled", fillcolor="lightgreen")
        dot.edge(source, target, label=action)

    # Render graph to file
    dot.render(output_path, format='png', cleanup=True)
    print(f"SID graph saved to: {output_path}.png")

In [ ]:
import os
from graphviz import Digraph

def draw_sbd_graph(parsed_sbd, output_dir):
    """
    Draws SBD graphs for each subject in parsed_sbd and saves them as PDFs.
    
    Parameters:
    - parsed_sbd: dict
        A dictionary where keys are subjects and values are lists of steps.
    - output_dir: str
        Directory where generated PDF files will be stored.
    """

    

    # Node styles for different state types
    state_styles = {
        'StartState': {"shape": "rectangle", "color": "yellow"},
        'EndState':   {"shape": "rectangle", "color": "yellow"},
        'SendState':  {"shape": "rectangle", "color": "green"},
        'ReceiveState': {"shape": "rectangle", "color": "pink"},
        'DoState':    {"shape": "rectangle", "color": "yellow"},
        'Unknown':    {"shape": "rectangle", "color": "yellow"},
    }

    for subject, steps in parsed_sbd.items():
        dot = Digraph(name=subject, format='pdf')
        dot.attr(rankdir='LR')

        # Step lookup map
        step_map = {step["num"]: step for step in steps}
        step_nums = sorted(step_map.keys())

        # Draw nodes
        for step in steps:
            label = step["description"]

            # Special case: StartState acting as ReceiveState
            if step["type"] == "StartState" and "From" in step and "Msg" in step and "receive" in step["description"].lower():
                style = state_styles["ReceiveState"]
            else:
                style = state_styles.get(step["type"], state_styles["Unknown"])

            dot.node(
                str(step["num"]),
                label=label,
                shape=style["shape"],
                style="filled",
                fillcolor=style["color"]
            )

        # Draw edges (transitions)
        for i, step in enumerate(steps):
            current_num = str(step["num"])

            if step["type"] == "EndState":
                continue  # EndState has no outgoing edges

            label = ""
            next_step_num = step_nums[i + 1] if i + 1 < len(step_nums) else None

            if step["type"] == "SendState":
                label = f"To:{step.get('To', '')}\\nMsg:{step.get('Msg', '')}"
            elif step["type"] == "ReceiveState":
                label = f"From:{step.get('From', '')}\\nMsg:{step.get('Msg', '')}"
            elif step["type"] == "DoState":
                label = step.get("Description", step["Description"])
            elif step["type"] == "StartState":
                # Handle From/To/Msg in StartState if present
                parts = []
                if "From" in step: parts.append(f"From:{step['From']}")
                if "To" in step: parts.append(f"To:{step['To']}")
                if "Msg" in step: parts.append(f"Msg:{step['Msg']}")
                label = "\\n".join(parts) if parts else step.get("Description", step.get("Action", step["description"]))

            if next_step_num:
                dot.edge(current_num, str(next_step_num), label=label)

        # Save as PDF
        safe_subject = subject.replace(" ", "_")
        filename = os.path.join(output_dir, f"SBD_{safe_subject}")
        dot.render(filename, cleanup=True)

    print(f"SBD graphs generated in: {output_dir}")

In [ ]:
from rdflib import Graph, Namespace, Literal  
from rdflib.namespace import RDF, RDFS, OWL, XSD
import re
from textwrap import dedent
from typing import Optional

# Namespaces
ABSTRACT = Namespace("http://www.imi.kit.edu/abstract-pass-ont#")
STANDARD = Namespace("http://www.i2pm.net/standard-pass-ont#")
BASE     = Namespace("http://subjective-me.jimdo.com/s-bpm/processmodels/2025-03-25/Page-1#")

def sid_to_pass_owl(mistral_text: str,
                    model_label: str = "PASS_Model",
                    out_file: Optional[str] = None) -> str:
    print("Inside OWL Function")

    # STEP 1: Parse Subjects and SID lines
    subjects = []
    sid_lines = []
    subj_mode, sid_mode = False, False
    for ln in mistral_text.splitlines():
        ln = ln.rstrip()
        if ln.startswith("### Subjects"):
            subj_mode, sid_mode = True, False
            continue
        if ln.startswith("### SID"):
            subj_mode, sid_mode = False, True
            continue
        if subj_mode and ln.startswith("-"):
            subjects.append(ln.lstrip("- ").strip())
        elif sid_mode and re.match(r"\d+\.", ln):
            sid_lines.append(ln)

    # STEP 2: Build RDF graph
    g = Graph(base=BASE)
    g.bind("abstract-pass-ont", ABSTRACT)
    g.bind("standard-pass-ont", STANDARD)
    g.bind("owl", OWL)
    g.bind("rdfs", RDFS)
    g.bind("xsd", XSD)

    # Add PASSProcessModel individual
    model_uri = BASE[model_label]
    g.add((model_uri, RDF.type, STANDARD.PASSProcessModel))
    g.add((model_uri, STANDARD.hasModelComponentID, Literal(f"{model_uri}#Model", datatype=XSD.string)))
    g.add((model_uri, STANDARD.hasModelComponentLabel, Literal(model_label, lang="en")))

    # Add ModelLayer SID_1
    sid_layer = BASE["SID_1"]
    g.add((sid_layer, RDF.type, ABSTRACT.ModelLayer))
    g.add((sid_layer, STANDARD.hasModelComponentID, Literal("SID_1", datatype=XSD.string)))
    g.add((sid_layer, STANDARD.hasModelComponentLabel, Literal("SID_1", lang="en")))
    g.add((sid_layer, STANDARD.hasPriorityNumber, Literal(1, datatype=XSD.positiveInteger)))
    g.add((model_uri, STANDARD.contains, sid_layer))

    # STEP 3: Subjects as FullySpecifiedSubject
    subj_id_map = {}
    for idx, subj_label in enumerate(subjects, start=2):
        sid = f"SID_1_FullySpecifiedSubject_{idx}"
        subj_uri = BASE[sid]
        subj_id_map[subj_label] = subj_uri

        g.add((subj_uri, RDF.type, STANDARD.FullySpecifiedSubject))
        g.add((subj_uri, STANDARD.hasModelComponentID, Literal(sid, datatype=XSD.string)))
        g.add((subj_uri, STANDARD.hasModelComponentLabel, Literal(subj_label, lang="en")))
        g.add((subj_uri, STANDARD.hasMaximumSubjectInstanceRestriction, Literal(1, datatype=XSD.integer)))
        g.add((subj_uri, ABSTRACT.hasExecutionCostPerHour, Literal(0.0, datatype=XSD.double)))

        g.add((sid_layer, STANDARD.contains, subj_uri))
        g.add((model_uri, STANDARD.contains, subj_uri))
    subj_id_map = {s.lower().strip(): uri for s, uri in subj_id_map.items()}

    # STEP 4: Process SID message lines
    mel_counter = 1
    msg_counter = 1
    for line in sid_lines:
        m = re.match(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)", line)
        if not m:
            continue
        sender, receiver, msg = m.groups()
        
        sender_uri   = subj_id_map[sender.strip().lower()]
        receiver_uri = subj_id_map[receiver.strip().lower()]
        

        msg_spec_id = f"SID_1_MessageSpecification_{msg_counter}"
        msg_spec_uri = BASE[msg_spec_id]
        g.add((msg_spec_uri, RDF.type, STANDARD.MessageSpecification))
        g.add((msg_spec_uri, STANDARD.hasModelComponentID, Literal(msg_spec_id, datatype=XSD.string)))
        g.add((msg_spec_uri, STANDARD.hasModelComponentLabel, Literal(msg, lang="en")))

        payload_id = f"PayloadDefinition_of_{msg_spec_id}"
        payload_uri = BASE[payload_id]
        g.add((payload_uri, RDF.type, OWL.Class))
        g.add((msg_spec_uri, STANDARD.containsPayloadDescription, payload_uri))

        mel_id = f"MessageExchangeList_on_SID_1_StandardMessageConnector_{mel_counter}"
        mel_uri = BASE[mel_id]
        conn_id = f"SID_1_StandardMessageConnector_{mel_counter}"
        conn_uri = BASE[conn_id]

        g.add((mel_uri, RDF.type, STANDARD.MessageExchangeList))
        g.add((mel_uri, STANDARD.hasModelComponentID, Literal(mel_id, datatype=XSD.string)))
        g.add((mel_uri, STANDARD.hasModelComponentLabel, Literal(conn_id, lang="en")))
        g.add((mel_uri, STANDARD.contains, msg_spec_uri))

        g.add((conn_uri, RDF.type, STANDARD.StandardMessageConnector))
        g.add((conn_uri, STANDARD.hasSender, sender_uri))
        g.add((conn_uri, STANDARD.hasReceiver, receiver_uri))
        g.add((conn_uri, STANDARD.hasMessageType, msg_spec_uri))
        g.add((mel_uri, STANDARD.contains, conn_uri))

        for parent in (sid_layer, model_uri):
            g.add((parent, STANDARD.contains, mel_uri))
            g.add((parent, STANDARD.contains, msg_spec_uri))
            g.add((parent, STANDARD.contains, conn_uri))

        mel_counter += 1
        msg_counter += 1
        
        
    # === Detect ### SBD section first ===
    sbd_text_lines = []
    in_sbd_section = False

    for ln in mistral_text.splitlines():
        ln_strip = ln.strip()

        # Start of SBD section
        if ln_strip.startswith("### SBD") or ln_strip.startswith("### Step 5"):
            in_sbd_section = True
            continue

        # Stop SBD when a new ### header starts (e.g., ### Process Model)
        if in_sbd_section and ln_strip.startswith("### "):
            break

        # Collect lines only if inside SBD section
        if in_sbd_section:
            sbd_text_lines.append(ln)

    # STEP 5: Parse MULTIPLE SBD SECTIONS
    # Collect all SBD blocks grouped by subject name
    sbd_sections = {}
    current_subject = None
    current_lines = []

    for ln in mistral_text.splitlines():
        ln_strip = ln.strip()
        # detect new subject block
        if ln_strip.startswith("#### "):
            # Save previous
            if current_subject and current_lines:
                sbd_sections[current_subject] = current_lines
            # start new
            current_subject = ln_strip.replace("####", "").replace(":", "").strip()
            current_lines = []
        elif current_subject:
            current_lines.append(ln)
    # Save last one
    if current_subject and current_lines:
        sbd_sections[current_subject] = current_lines

    
        # STEP 6: LOOP through EACH SBD section
    sbd_index = 1  # numbering SBDs
    for sbd_name, sbd_lines in sbd_sections.items():
        # Find which FullySpecifiedSubject this belongs to
        
        subj_uri = subj_id_map.get(sbd_name.lower().strip())
        if not subj_uri:
            print(f"Warning: No matching FullySpecifiedSubject for {sbd_name}, skipping SBD")
            continue

        # === Create SubjectBehavior individual ===
        
        subj_sid_idx = subj_uri.split("_")[-1]  # e.g. '2'
        sbd_id = f"SBD_{sbd_index}_SID_1_FullySpecifiedSubject_{subj_sid_idx}"
        sbd_uri = BASE[sbd_id]

        g.add((sbd_uri, RDF.type, STANDARD.SubjectBehavior))
        g.add((sbd_uri, STANDARD.hasModelComponentID, Literal(sbd_id, datatype=XSD.string)))
        g.add((sbd_uri, STANDARD.hasModelComponentLabel, Literal(f"SBD: {sbd_name}", lang="en")))
        g.add((sbd_uri, STANDARD.hasPriorityNumber, Literal(sbd_index, datatype=XSD.positiveInteger)))

        # Link SubjectBehavior to the FullySpecifiedSubject
        g.add((subj_uri, STANDARD.hasBehavior, sbd_uri))

        # Parse numbered state blocks
        sbd_blocks = []
        current_block = []
        for ln in sbd_lines:
            if re.match(r"\d+\.", ln.strip()):  # new block
                if current_block:
                    sbd_blocks.append(current_block)
                current_block = [ln.strip()]
            else:
                if current_block:
                    current_block.append(ln.strip())
        if current_block:
            sbd_blocks.append(current_block)

        # Extract states info
        send_state_info = {}
        receive_state_info = {}
        state_uris = []

        for idx, block in enumerate(sbd_blocks, start=1):
            m = re.match(r"\d+\.\s*(\w+State):\s*(.*)", block[0])
            if not m:
                continue
            state_type, label = m.groups()
            description_val, to_val, from_val, msg_val = None, None, None, None

            # Parse inner lines
            for line in block[1:]:
                if state_type == "SendState" and (m_to := re.match(r"To:\s*(.*)", line)):
                    to_val = m_to.group(1).strip()
                if state_type == "SendState" and (m_msg := re.match(r"Msg:\s*(.*)", line)):
                    msg_val = m_msg.group(1).strip()
                if state_type == "ReceiveState" and (m_from := re.match(r"From:\s*(.*)", line)):
                    from_val = m_from.group(1).strip()
                if state_type == "ReceiveState" and (m_msg := re.match(r"Msg:\s*(.*)", line)):
                    msg_val = m_msg.group(1).strip()
                if state_type in ["StartState", "DoState"] and (m_desc := re.match(r"Description:\s*(.*)", line)):
                    description_val = m_desc.group(1).strip()

            # Store send/receive info
            if state_type == "SendState":
                send_state_info[idx] = {"to": to_val, "msg": msg_val}
            if state_type == "ReceiveState":
                receive_state_info[idx] = {"from": from_val, "msg": msg_val}

            # Add State individual with proper SBD prefix
            state_id = f"SBD_{sbd_index}_{state_type}_{idx}"
            state_uri = BASE[state_id]
            state_uris.append((state_type, idx, label, description_val))

            g.add((state_uri, RDF.type, STANDARD[state_type]))
            g.add((state_uri, STANDARD.hasModelComponentID, Literal(state_id, datatype=XSD.string)))
            g.add((state_uri, STANDARD.hasModelComponentLabel, Literal(label, lang="en")))

            # Contain states inside the SubjectBehavior
            g.add((sbd_uri, STANDARD.contains, state_uri))

        # Create transitions between states (inside the same SBD)
        transition_counter = 1
        for i in range(len(state_uris) - 1):
            source_type, source_idx, _, source_desc = state_uris[i]
            target_type, target_idx, _, _ = state_uris[i + 1]

            trans_id = f"SBD_{sbd_index}_{source_type}Transition_{transition_counter}"
            trans_uri = BASE[trans_id]

            # Choose transition type
            if source_type == "SendState":
                trans_type = STANDARD.SendTransition
                info = send_state_info.get(source_idx, {})
                trans_label = f"To: {info.get('to', 'Unknown')}\nMsg: {info.get('msg', 'Unknown')}"
            elif source_type == "ReceiveState":
                trans_type = STANDARD.ReceiveTransition
                info = receive_state_info.get(source_idx, {})
                trans_label = f"From: {info.get('from', 'Unknown')}\nMsg: {info.get('msg', 'Unknown')}"
            else:
                trans_type = STANDARD.DoTransition
                trans_label = source_desc if source_desc else "Continue Process"

            g.add((trans_uri, RDF.type, trans_type))
            g.add((trans_uri, STANDARD.hasModelComponentID, Literal(trans_id, datatype=XSD.string)))
            g.add((trans_uri, STANDARD.hasModelComponentLabel, Literal(trans_label, lang="en")))

            source_uri = BASE[f"SBD_{sbd_index}_{source_type}_{source_idx}"]
            target_uri = BASE[f"SBD_{sbd_index}_{target_type}_{target_idx}"]

            g.add((trans_uri, STANDARD.hasSourceState, source_uri))
            g.add((trans_uri, STANDARD.hasTargetState, target_uri))

            # Transitions also belong only inside the same SubjectBehavior
            g.add((sbd_uri, STANDARD.contains, trans_uri))

            transition_counter += 1

        # Link the SubjectBehavior itself to model & SID layer
        g.add((sid_layer, STANDARD.contains, sbd_uri))
        g.add((model_uri, STANDARD.contains, sbd_uri))

        sbd_index += 1

    
    # example class
    dm_class = BASE["VisioShapesInternalDataMappingFunction"]
    g.add((dm_class, RDF.type, OWL.Class))
    g.add((dm_class, RDFS.subClassOf, STANDARD.DataMappingFunction))

    # STEP 7: Serialize RDF/XML
    xml_body = g.serialize(format="application/rdf+xml")

    entities = dedent("""\
        <!DOCTYPE rdf:RDF [
            <!ENTITY owl "http://www.w3.org/2002/07/owl#" >
            <!ENTITY xsd "http://www.w3.org/2001/XMLSchema#" >
            <!ENTITY rdfs "http://www.w3.org/2000/01/rdf-schema#" >
            <!ENTITY abstract-pass-ont "http://www.imi.kit.edu/abstract-pass-ont#" >
            <!ENTITY standard-pass-ont "http://www.i2pm.net/standard-pass-ont#" >
            <!ENTITY rdf "http://www.w3.org/1999/02/22-rdf-syntax-ns#" >
        ]>
    """)
    xml_body_nohead = "\n".join(xml_body.splitlines()[1:])
    final_xml = f'<?xml version="1.0"?>\n{entities}\n{xml_body_nohead}'

    if out_file:
        with open(out_file, "w", encoding="utf-8") as f:
            f.write(final_xml)

    return final_xml

In [ ]:
import panel as pn 
from openai import OpenAI

# Add a Panel pane to display the image
image_view = pn.pane.PNG(width=700)

pn.extension()

# --- API setup ---
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "Llama-3.3-70B"
client = OpenAI(api_key=api_key, base_url=base_url)

# --- Prompt Templates ---
extract_subjects_prompt = """
You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Given the following scenario:

\"\"\"{scenario}\"\"\"


Only identify all the subjects involved.  
Return as a simple bullet list using "-":

### Subjects:
- Subject 1
- Subject 2
- ...
"""

sid_sbd_prompt = """
You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Analyze the following scenario step-by-step and produce both the Subject Interaction Diagram (SID) and Subject Behavior Diagrams (SBDs).

\"\"\"{scenario}\"\"\"

### Step 1: Identify Subjects

Finalized subjects:
{subjects}

Important: Use ONLY these subjects. Do NOT introduce any other subjects. 
All interactions in the SID must be between these subjects.

Very Strict Rule:
- You MUST use ONLY these subjects EXACTLY as written (same spelling & capitalization).
- DO NOT introduce, rename, or re-add any subject (like "Payment Gateway" or "Website").
- If a subject seems missing, map its role to one of the listed subjects.

### Step 2: Identify Messages (Noun Form Only)

List key messages exchanged between subjects using **noun phrases only** (e.g., “Order”, “Inventory Status” etc.).

## Step 3: Subject Interaction Diagram (SID)

Write the SID as a **numbered list** describing subject-to-subject message flows.

Format:
'[Subject A] -> [Subject B]: [Message]'

### Step 4: Think Step-by-Step (REASONING PHASE)

- Break down the subject's journey into clear events.
- Identify each type of PASS state involved.
- Consider conditional branches (e.g., item in stock vs out of stock).
- Match each message in the SID to the appropriate state.
- Ensure logical transitions and valid branches or GotoSteps.
-When the subject receives a Declination Message (e.g. item out of stock), the flow must proceed to a 
 DoState: Decide upon further action, where the subject either cancels the order or chooses another supplier. 
 This ensures proper branching and no immediate sending of cancellation message without decision.
 
-When the subject receives e.g. a payment request, they must first send payment, then receive payment confirmation, 
and then transition to the final state (e.g., Order Confirmed).

-The Order Declination branch must go to a DoState (decision) and must not pass through payment states.

-Never allow decision branches to lead to states meant for a different outcome (e.g., do not send to "Receive Payment Confirmation" if no payment was sent).

**Please write out your reasoning step-by-step before generating the Subject Behavior Diagram (SBD).**

### Step 5: Subject Behavior Diagram (SBD)

For each subject from Step 1, describe its internal behavior using PASS state types:

- **StartState**, **DoState**, **SendState**, **ReceiveState**, **EndState**

**Rules**:
- 'SendState' :- include 'To:' and 'Msg:'
- 'ReceiveState' :- include 'From:' and 'Msg:'
- Use 'Branches:' for decisions
  Always increment step numbers correctly. Double-check that Branches: refer to valid steps (e.g., "Step: 10", not "Step: 9" if Step 9 is an EndState).
- Use GotoStep: to return to earlier steps (e.g., sending new order again)
-For retry paths (like choosing another supplier), always use GotoStep: instead of duplicating behaviour"
-Ensure branches reference the correct step numbers

For every **DoState**, include a Description: field that outlines the internal logic or condition that leads to the next state. This should capture possible outcomes necessary to exit the DoState.

- In decision branches:
  - Each `Step:` must point to a valid numbered step that **actually exists**.
  - If retrying earlier steps, use `GotoStep:` and **refer to original step number** (e.g., GotoStep: 2).
- Do not invent new EndStates or GotoSteps that aren't clearly part of a branch.

Example Branch:

6. DoState: Decide next action
   Branches:
   - Step: 7
     Description: Cancel order
   - Step: 9
     Description: Try another supplier

7. SendState: Send Cancellation
8. EndState: Order Cancelled
9. GotoStep: 2  # Retry order

**Important Note on Step Numbering and 'Next:' References:**

- Each step number must be sequential unless explicitly branching or looping back.
- The 'Next:' field in each state should refer to the **very next logical step in the process**, not skip decision points or intermediate steps.
- If a decision is required next, 'Next:' should point to the corresponding DoState step, not jump directly to an EndState or unrelated step.
- Only use 'Next: EndState' when the flow truly terminates at that step. And in 'Next: EndState', put EndState
  number like 'Next: 8) if 8.EndState
- Avoid skipping over intermediate steps such as decisions or sends; maintain logical process flow.

- After receiving confirmations or important messages, flow should proceed to the next logical decision or processing step, not directly to an EndState unless the process actually terminates.

****Important Notes on EndStates:
Use separate EndStates for different outcomes:
e.g., one for “Successfully Completed”, another for “Cancelled”.
Do not route cancellation, decline, or failed attempts to an EndState meant for successful completions.
If a common EndState is used for multiple paths, name it generically, e.g., “Process Completed” or “End of Process”.
Ensure Next: fields route only logically to the correct EndState matching the path.

Model each subject one by one.

### Step 6: Check Logic and Flow

Ensure:
- Each decision branch leads to an `EndState` or valid `GotoStep:`
- No duplicate messages
- Each ReceiveState handles only mutually exclusive messages

### Example Scenario (Healthcare):
A patient books an appointment using a healthcare app.  
The app sends the appointment request to the hospital system.  
The hospital system checks the doctor’s availability.  
If the doctor is available, it confirms the appointment.  
The app then sends a confirmation message to the patient.
If the doctor is not available, hospital system declines the appointment.
The patient is sent decline message.
The patient then decides upon further action either the patient cancels request and all ends
or the patient chooses another hospital.

### SBD:
####Patient:
1. StartState: Decide to make appointment  
   Description: Patient initiates appointment process

2. SendState: Send Appointment Request to Healthcare App  
   To: Healthcare App  
   Msg: Appointment Request

3. ReceiveState: Receive Reply from Healthcare App  
   Choices:
   - From: Healthcare App  
     Msg: Appointment Confirmation  
     Next: 4  
   - From: Healthcare App  
     Msg: Appointment Declination  
     Next: 5  

4. EndState: Appointment booked

5. DoState: Decide upon further action  
   Branches:
   - Step: 6  
     Description: Cancel appointment request  
   - Step: 8  
     Description: Choose another hospital and try again

6. DoState: Cancel appointment request  
   Description: Cancel

7. EndState: Appointment request cancelled (end of this path)
8. GotoStep: 2  # Retry appointment loop

(Similar steps would follow for Healthcare App and Hospital System…)
Now, do the same for the scenario provided above. Output the **SID first**, followed by **SBDs for each subject**.
Model all subjects from Step 1. Do not skip the last subject. Ensure each subject has its full SBD.
Repeat Steps 4–5 for **every subject** listed in Step 1. 
Do not skip subjects 
Ensure each has a complete behavior diagram from StartState to EndState.
"""

# --- Panel Widgets ---
scenario_input = pn.widgets.TextAreaInput(name='PASS Scenario', height=150, width=700)
extract_btn = pn.widgets.Button(name="Extract Subjects", button_type='primary')
generate_btn = pn.widgets.Button(name="Generate SID & SBD", button_type='success', disabled=True)

subjects_box = pn.widgets.CheckBoxGroup(name="Subjects", options=[], inline=False)
add_subject_input = pn.widgets.TextInput(name="Add New Subject", placeholder="e.g. Inventory System")
add_subject_btn = pn.widgets.Button(name="Add Subject", button_type='primary')
rename_input = pn.widgets.TextInput(name="Rename/Merge Subjects (comma separated)", placeholder="e.g. Website, App -> Customer Portal")
apply_merge_btn = pn.widgets.Button(name="Apply Merge/Rename", button_type='warning')

output_markdown = pn.pane.Markdown("### Output will appear here...", sizing_mode="stretch_width")

# --- Callbacks ---

def extract_subjects(event):
    scenario = scenario_input.value.strip()
    if not scenario:
        output_markdown.object = "Please enter a scenario description."
        return
    
    prompt = extract_subjects_prompt.replace("{scenario}", scenario)
    try:
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
            temperature=0.3,
        )
        raw_subjects = response.choices[0].message.content.strip()
        # Parse subjects (lines starting with "- ")
        subjects = [line[2:].strip() for line in raw_subjects.splitlines() if line.strip().startswith("- ")]
        subjects_box.options = subjects
        subjects_box.value = subjects  # pre-select all
        generate_btn.disabled = False
        output_markdown.object = f"### Extracted Subjects:\n{raw_subjects}"
    except Exception as e:
        output_markdown.object = f"API Error: {e}"

def apply_merge(event):
    
    merge_text = rename_input.value.strip()
    if "->" not in merge_text:
        return
    left, right = merge_text.split("->")
    old = [s.strip() for s in left.split(",")]
    new = right.strip()
    updated = [new if s in old else s for s in subjects_box.options]
    subjects_box.options = list(dict.fromkeys(updated))  # remove duplicates
    subjects_box.value = subjects_box.options
    
def add_subject(event):
    new_subject = add_subject_input.value.strip()
    if not new_subject:
        return
    # Avoid duplicates
    if new_subject not in subjects_box.options:
        subjects_box.options = subjects_box.options + [new_subject]
        # Optionally, auto-check the new subject
        subjects_box.value = subjects_box.value + [new_subject]
    add_subject_input.value = ""  # clear input


def generate_sid_sbd(event):
    scenario = scenario_input.value.strip()
    finalized_subjects = "\n".join(f"- {s}" for s in subjects_box.value)
    prompt = sid_sbd_prompt.replace("{scenario}", scenario).replace("{subjects}", finalized_subjects)
    
    try:
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
            temperature=0.5
        )
        sid_sbd = response.choices[0].message.content.strip()
        print("############ API RESPONSE #############\n", sid_sbd)
        
        # Build clean text for OWL function
        llm_text = (
            f"### Subjects\n{finalized_subjects}\n\n"
            f"### SID\n{sid_sbd}"
        )

        
        
        output_markdown.object = (
                                  f"### Subjects:\n{finalized_subjects}\n\n"
                                  f"### SID & SBD Output:\n\n```\n{sid_sbd}\n```"
                                  )
        
         # --- Graph Drawing ---
        sid_pairs = parse_sid(sid_sbd)
        process = {
                    "SID": sid_pairs
                  }
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        graph_path = fr"C:\Users\qwerz\Documents\Privat\WWU\Wirtschaftsinformatik\Semester 7\Bachelorarbeit\PASS Diagrams\Diagram\llama\SID-{timestamp}"
        print("Graph saved at:", graph_path + ".png")
        draw_sid_graph(process["SID"],output_path=graph_path)
        
        import os
        print("Exists:", os.path.exists(graph_path + ".png"))

        
        with open(graph_path + ".png", "rb") as f:
                 image_view.object = f.read()
        
        
        sbd_text = extract_sbd_section(sid_sbd)
        parsed_sbd = parse_sbd(sbd_text)
        sbd_path = fr"C:\Users\qwerz\Documents\Privat\WWU\Wirtschaftsinformatik\Semester 7\Bachelorarbeit\PASS Diagrams\Diagram\llama"

        draw_sbd_graph(parsed_sbd,output_dir=sbd_path)
        
        
        print("llm_text",llm_text)
        xml = sid_to_pass_owl(llm_text, out_file=r"C:\Users\qwerz\Documents\Privat\WWU\Wirtschaftsinformatik\Semester 7\Bachelorarbeit\PASS Diagrams\Diagram\llama\OWLFile.owl")
        
    except Exception as e:
        output_markdown.object += f"\n\n Error: {e}"

# --- Button bindings ---
extract_btn.on_click(extract_subjects)
add_subject_btn.on_click(add_subject)
apply_merge_btn.on_click(apply_merge)
generate_btn.on_click(generate_sid_sbd)

# --- Layout ---
app = pn.Column(
    pn.pane.Markdown("# PASS Process Modeling with LLM"),
    scenario_input,
    pn.Row(extract_btn, generate_btn),
    subjects_box,
    pn.Row(add_subject_input, add_subject_btn),
    pn.Row(rename_input, apply_merge_btn),
    output_markdown,
    pn.pane.Markdown("### SID Graph:"),
    image_view,
    sizing_mode="stretch_width"
)

app.show()